## RDKIT Molecular Fingerprint Matrices
#### ALL DATABASES ACCESSED 12/2019
##### Author : Eryk Kropiwnicki | eryk.kropiwnicki@icahn.mssm.edu

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit import Avalon
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D,Generate
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import os

In [2]:
os.chdir('../scripts')
from export_script import *
os.chdir('../RDKIT')

#### Input file : drug_metadata.csv (master drug list generated from Drug Metadata Aggregation.ipynb)

In [3]:
df = pd.read_csv('../mapping_files/merged_drug_metadata.tsv', sep = '\t')

In [4]:
df = df.drop(columns = ['Synonyms','PubChemID','InChiKey','Molecular_formula'])

In [5]:
df.head()

,Drug_name,Canonical_SMILES
0,goserelin,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NN...
1,cetrorelix,CC(C)CC(C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NC...
2,ciclosporin,CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(...
3,octreotide,CC(C1C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=...
4,choline,C[N+](C)(C)CCO


In [6]:
molecule = [Chem.MolFromSmiles(x) for x in df['Canonical_SMILES']]
df.loc[:,'Molecule'] = pd.Series(molecule, index=df.index)

RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [17:44:05] WARNING: not r

In [7]:
df = df.dropna()

In [8]:
all_drugs = list(df['Drug_name'])

### Morgan Fingerprints

In [9]:
# change radius and useFeatures = True as needed 
info = {}
morg_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,bitInfo=info) for x in df['Molecule']]

In [10]:
# Converting bit vectors into binary array 
morg_np_fps = []
for fp in morg_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    morg_np_fps.append(arr)
    
morg_df = pd.DataFrame(morg_np_fps, index = all_drugs)

In [11]:
column_labels = []
for col in morg_df.columns:
    column_labels.append('Morgan_' + str(col))
morg_df.columns = column_labels

In [12]:
morg_df.head()

,Morgan_0,Morgan_1,Morgan_2,Morgan_3,Morgan_4,Morgan_5,Morgan_6,Morgan_7,Morgan_8,Morgan_9,...,Morgan_2038,Morgan_2039,Morgan_2040,Morgan_2041,Morgan_2042,Morgan_2043,Morgan_2044,Morgan_2045,Morgan_2046,Morgan_2047
aliskiren,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ntbc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
piperacillin,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
lymecycline,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cisplatin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
morg_df.to_csv('../data/RDKIT/RDKIT_morgan_fp_matrix.tsv', sep = '\t')

### Avalon Fingerprints

In [14]:
avalon_fps = [Avalon.pyAvalonTools.GetAvalonFP(x) for x in df['Molecule']]

In [15]:
avalon_np_fps = []
for fp in avalon_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    avalon_np_fps.append(arr)

avalon_df = pd.DataFrame(avalon_np_fps, index = all_drugs)

In [16]:
column_labels = []
for col in avalon_df.columns:
    column_labels.append('Avalon_' + str(col))
avalon_df.columns = column_labels

In [17]:
avalon_df.head()

,Avalon_0,Avalon_1,Avalon_2,Avalon_3,Avalon_4,Avalon_5,Avalon_6,Avalon_7,Avalon_8,Avalon_9,...,Avalon_502,Avalon_503,Avalon_504,Avalon_505,Avalon_506,Avalon_507,Avalon_508,Avalon_509,Avalon_510,Avalon_511
aliskiren,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
ntbc,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
piperacillin,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
lymecycline,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
cisplatin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
avalon_df.to_csv('../data/RDKIT/RDKIT_avalon_fp_matrix.tsv', sep = '\t')

### Atom Pair Fingerprints

In [19]:
atom_pairs_fps = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(x) for x in df['Molecule']]

In [20]:
ap_np_fps = []
for fp in atom_pairs_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    ap_np_fps.append(arr)

ap_df = pd.DataFrame(ap_np_fps, index = all_drugs)

In [21]:
column_labels = []
for col in ap_df.columns:
    column_labels.append('AtomPair_' + str(col))
ap_df.columns = column_labels

In [22]:
ap_df.head()

,AtomPair_0,AtomPair_1,AtomPair_2,AtomPair_3,AtomPair_4,AtomPair_5,AtomPair_6,AtomPair_7,AtomPair_8,AtomPair_9,...,AtomPair_2038,AtomPair_2039,AtomPair_2040,AtomPair_2041,AtomPair_2042,AtomPair_2043,AtomPair_2044,AtomPair_2045,AtomPair_2046,AtomPair_2047
aliskiren,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ntbc,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
piperacillin,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
lymecycline,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cisplatin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
ap_df.to_csv('../data/RDKIT/RDKIT_atompair_fp_matrix.tsv', sep = '\t')

### Topological Fingerprints

In [24]:
top_fps = [FingerprintMols.FingerprintMol(x , minPath = 1,
                                          maxPath = 7, fpSize = 2048, bitsPerHash = 2,
                                          useHs = True, tgtDensity = 0, minSize = 128) for x in df['Molecule']]


In [25]:
top_np_fps = []
for fp in top_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    top_np_fps.append(arr)

top_df = pd.DataFrame(top_np_fps, index = all_drugs)

In [26]:
column_labels = []
for col in top_df.columns:
    column_labels.append('Topological_' + str(col))
top_df.columns = column_labels

In [27]:
top_df.head()

,Topological_0,Topological_1,Topological_2,Topological_3,Topological_4,Topological_5,Topological_6,Topological_7,Topological_8,Topological_9,...,Topological_2038,Topological_2039,Topological_2040,Topological_2041,Topological_2042,Topological_2043,Topological_2044,Topological_2045,Topological_2046,Topological_2047
aliskiren,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
ntbc,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
piperacillin,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
lymecycline,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
cisplatin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
top_df.to_csv('../data/RDKIT/RDKIT_topological_fp_matrix.tsv', sep = '\t')

### RDKit Fingerprints

In [ ]:
rdk_fps = [Chem.RDKFingerprint(x,maxPath=2) for x in df['Molecule']]

In [ ]:
rdk_np_fps = []
for fp in rdk_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    rdk_np_fps.append(arr)

rdk_df = pd.DataFrame(rdk_np_fps, index = all_drugs)

In [ ]:
column_labels = []
for col in rdk_df.columns:
    column_labels.append('RDKf2_' + str(col))
rdk_df.columns = column_labels

In [ ]:
rdk_df.to_csv('../data/RDKIT/RDKIT_rdkit_fp_matrix.tsv', sep = '\t')